In [17]:
import os
import json
from PIL import Image
import torch
from torch.utils.data import Dataset
from torchvision import transforms
import csv

# 커스텀 데이터셋 클래스 정의
class ShipDataset(Dataset):
    def __init__(self, csv_file, transform=None):
        self.data = self.load_csv(csv_file)  # CSV 파일에서 이미지와 JSON 경로 불러오기
        self.transform = transform

    def load_csv(self, csv_file):
        data = []
        with open(csv_file, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            next(reader)  # 헤더 스킵
            for row in reader:
                img_path, json_path = row
                data.append((img_path, json_path))
        return data

    def get_bbox_and_angle(self, json_file):
        with open(json_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        cx, cy, width, height, angle = [], [], [], [], []
        if 'features' in data:
            for feature in data['features']:
                if 'properties' in feature:
                    # JSON에서 cx, cy, width, height, angle 값을 모두 가져오기
                    cx.append(feature['properties'].get('cx', 0))
                    cy.append(feature['properties'].get('cy', 0))
                    width.append(feature['properties'].get('width', 0))
                    height.append(feature['properties'].get('height', 0))
                    angle.append(feature['properties'].get('object_angle', 0.0))

        return cx, cy, width, height, angle

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_path, json_path = self.data[idx]

        # 이미지 불러오기
        image = Image.open(img_path).convert("RGB")
        
        # JSON에서 모든 객체의 BBox 및 각도 정보 추출
        cx, cy, width, height, angle = self.get_bbox_and_angle(json_path)

        # 전처리 적용
        if self.transform:
            image = self.transform(image)
        
        # 모든 객체의 타겟 값 리스트로 묶기
        targets = []
        for i in range(len(cx)):
            targets.append([cx[i], cy[i], width[i], height[i], angle[i]])

        # 타겟을 텐서로 변환
        targets = torch.tensor(targets, dtype=torch.float32)

        return image, targets

# 전처리 정의
transform = transforms.Compose([
    transforms.Resize((256, 256)),  # 이미지 크기 조정
    transforms.ToTensor(),          # 이미지 텐서로 변환
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # 이미지 정규화
])

# 데이터셋 생성
csv_file = 'train.csv'  # 이미지 경로와 JSON 경로가 담긴 CSV 파일
dataset = ShipDataset(csv_file, transform=transform)

# 데이터셋 확인
image, targets = dataset[0]  # 첫 번째 샘플 불러오기
print(f"Image Shape: {image.shape}")
print(f"Targets (cx, cy, width, height, angle) for all objects: {targets}")

Image Shape: torch.Size([3, 256, 256])
Targets (cx, cy, width, height, angle) for all objects: tensor([[ 518.9017,  505.1197,  216.3758,  192.7043, -130.2250],
        [ 493.5748,  480.6295,   19.7542,   19.9513,  -42.6155]])


In [18]:
import torch
import torch.nn as nn

# 간단한 모델 정의 (여러 객체에 대한 cx, cy, width, height, angle 예측)
class SimpleModel(nn.Module):
    def __init__(self, num_objects=10):
        super(SimpleModel, self).__init__()
        self.fc = nn.Linear(256 * 256 * 3, num_objects * 5)  # (cx, cy, width, height, angle) * num_objects 예측

    def forward(self, x):
        x = x.view(x.size(0), -1)  # 이미지를 Flatten
        out = self.fc(x)
        return out.view(x.size(0), -1, 5)  # (batch_size, num_objects, 5)

# 모델 생성
model = SimpleModel(num_objects=10)  # 최대 10개의 객체를 예측한다고 가정

# 손실 함수 정의 (MSELoss 사용)
loss_fn = nn.MSELoss()

# 가짜 입력 데이터 (배치 크기 2, 이미지 크기 256x256, 3채널)
inputs = torch.randn(2, 3, 256, 256)

# 가짜 Ground Truth (각 이미지당 2개의 객체 예시)
true_targets = torch.tensor([[[110.5, 115.3, 50.0, 75.0, 45.0], [210.3, 115.1, 50.2, 80.2, 90.0]],
                             [[150.0, 125.0, 60.0, 85.0, 30.0], [250.0, 135.0, 40.0, 90.0, 60.0]]])

# 모델 예측
pred_targets = model(inputs)

# 손실 계산
loss = loss_fn(pred_targets, true_targets)
print(f"Loss: {loss.item()}")


Loss: 5193.89208984375


c:\ProgramData\anaconda3\envs\dust\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([2])) that is different to the input size (torch.Size([2, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
